In [1]:
import os
import torch


from main import get_model
from dataset.dataset import LoaderConstructor, create_alicewonderland_dataset, create_rocstories_dataset

from datasets import load_dataset


/home/raquelpanadero/.pyenv/versions/3.11.0/envs/ml-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = {
    'dataset': 'rocstories',
    'batch_size': 3,
    'max_length': 20,
    'embed_dim': 512,
    'min_text_length': 100, 
    'tokenizer': 'torchtext'
}

# Load the dataset
if "wikitest" in cfg['dataset']:
    dataset = load_dataset("wikitext", f"{cfg['dataset']}-raw-v1")
    for split in dataset.keys():
        dataset[split] = dataset[split].filter(
            lambda x: len(x["text"]) > cfg.min_text_length
        )

elif cfg['dataset'] == "rocstories":
    dataset = create_rocstories_dataset(os.getcwd())
elif cfg['dataset'] == "alicewonderland":
    dataset = create_alicewonderland_dataset(os.getcwd())

# Construct the dataloaders
lc = LoaderConstructor(
    dataset=dataset,
    batch_size=cfg['batch_size'],
    max_length=cfg['max_length'],
    tokenizer_type=cfg['tokenizer'],
    labels_sequence=False,
)
loaders = {}
for loader in ["train", "validation", "test"]:
    loaders[loader] = lc.construct_loader(split=loader)

input_size = loaders["train"].dataset.input_size
vocab_size = lc.vocab_size
output_size = lc.output_size
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def initialize_model(model_name):
    model = get_model(
        model=model_name,
        vocab_size=vocab_size,
        embed_dim=cfg['embed_dim'],
        seq_len=input_size,
        output_dim=output_size,
        device=device,
    )

    model_weights = {
        'lstm': f"trained_models/lstm_{cfg['dataset']}_lr=0_0001_lastepoch.pt",
        'transformer': f"trained_models/transformer_{cfg['dataset']}_lr=0_0001_lastepoch.pt",
        'xlstm': f"trained_models/xlstm_{cfg['dataset']}_lr=0_0001_lastepoch.pt",
    }

    model.load_state_dict(torch.load(model_weights[model_name]))
    model.to(device).eval()
    return model

In [47]:
def evaluate(model, batch, test=False, n_next_words=5):    
    # Forward pass
    inputs, labels = batch["input_ids"].to(device), batch["labels"].to(device)
    labels = labels.contiguous().view(-1)
    output = model(inputs).view(-1, output_size)

    for i in range(batch["input_ids"].shape[0]):
        input_text = lc.tokenizer.decode(inputs[i].tolist(), target=False)
        target_text = lc.tokenizer.decode(labels[i].unsqueeze(0).tolist(), target=True)[0]
        predicted_text = lc.tokenizer.decode(torch.argmax(output[i], dim=-1).unsqueeze(0).tolist(), target=True)[0]
        print(f"Input sentence: {' '.join(input_text)}")
        print(
            f"Target sentence: {target_text}"
        )
        print(
            f"Predicted sentence: {predicted_text}"
        )
        print()
        
    # Predict next words
    if test:
        mini_dataset = [
            {"text": "She was an extremely fast runner, she went to the forest every day to practice and be", "predictions": []},
            {"text": "The sun went down, making long shadows as the kids played by the big old tree when her mum", "predictions": []},
            {"text": "I would have gotten the new job, but my work wasn't good", "predictions": []},
            {"text": "She read a bedtime story with her parents and then went to sleep to rest for the next", "predictions": []},
            {"text": "Jason didn't understand why his parents wouldn't let him buy his favourite", "predictions": []},
            {"text": "Verena is the most beautiful girl I have ever seen. I love her blue eyes", "predictions": []},
        ]

        # Tokenize the texts
        tokenised_samples = lc.tokenizer.create_tokens(mini_dataset)
        tokenised_samples = lc.tokenizer.pad_sequences(tokenised_samples)
        encodings = lc.tokenizer.encode(tokenised_samples)

        # No labels in this case
        inputs = encodings[:, 1:].to(device)
        decoded_inputs = [lc.tokenizer.decode(sample.tolist(), target=False) for sample in inputs]

        with torch.no_grad():
            for _ in range(n_next_words):
                output = model(inputs)
                next_token = torch.argmax(output, dim=-1)

                # Decode the tokens to get the words
                decoded_tokens = lc.tokenizer.decode(next_token.tolist(), target=True)

                # Encode the tokens to get the input for the next iteration
                encoded_tokens = lc.tokenizer.encode([decoded_tokens]).to(device)

                # Concatenate the input with the new tokens
                inputs = torch.cat([inputs[:, 1:], encoded_tokens.reshape(-1, 1)], dim=1)
                
                for i, token in enumerate(decoded_tokens):
                    mini_dataset[i]["predictions"].append(token)

        for sample, input_sentence in zip(mini_dataset, decoded_inputs):
            print(f"Input sentence: {' '.join(input_sentence)}")
            print(f"Predicted next words: {' '.join(sample['predictions'])}")
            print()


In [44]:
batch = next(iter(loaders["train"]))
for model_name in ["xlstm", "lstm", "transformer"]:
    print('*' * 50)
    print(f"Evaluating model: {model_name}")
    print('*' * 50)
    model = initialize_model(model_name)
    evaluate(model, batch)
    print('*' * 50)

**************************************************
Evaluating model: xlstm
**************************************************
Input sentence: kate met a guy in a club they talked for a while in the back of the club the dj
Target sentence: played
Predicted sentence: played

Input sentence: <oov> ruth could hit more <oov> than anyone <oov> win games he joined the yankees he still hit more than
Target sentence: anybody
Predicted sentence: anybody

Input sentence: suzy got a new phone she never thought she would be getting an iphone but it was cheaper than normal
Target sentence: so
Predicted sentence: so

**************************************************
**************************************************
Evaluating model: lstm
**************************************************
Input sentence: kate met a guy in a club they talked for a while in the back of the club the dj
Target sentence: played
Predicted sentence: were

Input sentence: <oov> ruth could hit more <oov> than anyone <oov> wi

In [48]:
batch = next(iter(loaders["test"]))
for model_name in ["xlstm", "lstm", "transformer"]:
    print('*' * 50)
    print(f"Evaluating model: {model_name}")
    print('*' * 50)
    model = initialize_model(model_name)
    evaluate(model, batch, test=True, n_next_words=5)
    print('*' * 50)

**************************************************
Evaluating model: xlstm
**************************************************
Input sentence: i recently installed a new script on my computer it helps me see what i ve earned in a day
Target sentence: when
Predicted sentence: of

Input sentence: the man punched a woman the woman fell down neighbors called the cops the man went to jail the woman
Target sentence: filed
Predicted sentence: noticed

Input sentence: town to get his shoes shined when he arrived he got his shoes polished and cleaned on his way home
Target sentence: someone
Predicted sentence: from

Input sentence: <pad> <pad> <pad> she was an extremely fast runner she went to the forest every day to practice and be
Predicted next words: a new one she was

Input sentence: <pad> the sun went down making long shadows as the kids played by the big old tree when her mum
Predicted next words: was afraid the baby was

Input sentence: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> i w